# Malware Image Classification 

Simple Classifier를 통해 Malware Image를 분류해보자.

* 이전에 작성했던 코드(github)를 참고해도 좋고
* 선대 연구자들이 연구한 자료 또는 코드를 참고해도 좋다.
* 뭐가 되도 좋으니 하나만 한 번 제대로 해보자!

## # 1. Import Library & Load Data

In [1]:
import os
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
with open('../data.pkl', 'rb') as f:
    data = pickle.load(f)  # pickle.load()
    

불러온 데이터를 확인해본다.

In [3]:
data

(array([[ 94.,  32.,   2., ..., 129., 102., 124.],
        [ 54.,  29.,   6., ..., 106., 119., 100.],
        [ 54.,  29.,   6., ..., 120., 126., 104.],
        ...,
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.],
        [ 65.,  22.,  59., ..., 117., 116.,  98.]]),
 array([ 0.,  0.,  0., ..., 24., 24., 24.]),
 ['Adialer.C',
  'Agent.FYI',
  'Allaple.A',
  'Allaple.L',
  'Alueron.gen!J',
  'Autorun.K',
  'C2LOP.gen!g',
  'C2LOP.P',
  'Dialplatform.B',
  'Dontovo.A',
  'Fakerean',
  'Instantaccess',
  'Lolyda.AA1',
  'Lolyda.AA2',
  'Lolyda.AA3',
  'Lolyda.AT',
  'Malex.gen!J',
  'Obfuscator.AD',
  'Rbot!gen',
  'Skintrim.N',
  'Swizzor.gen!E',
  'Swizzor.gen!I',
  'VB.AT',
  'Wintrim.BX',
  'Yuner.A'],
 [122,
  116,
  2949,
  1591,
  198,
  106,
  200,
  146,
  177,
  162,
  381,
  431,
  213,
  184,
  123,
  159,
  136,
  142,
  158,
  80,
  128,
  132,
  408,
  97,
  800])

In [4]:
data[0]

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [5]:
malimg[1]

NameError: name 'malimg' is not defined

In [ ]:
malimg[1]

In [ ]:
malimg[2]

In [ ]:
malimg[3]

### Import Torch Library

In [4]:
# 참고(Reference)
# 나의 깃헙
# https://github.com/Steve-YJ/deep-learning-from-scratch-studying/blob/master/02_PyTorch_Introduction_Basic/Ch05_5.5_%EC%99%80%EC%9D%B8%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb


# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable  # Variable?
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn 라이브러리 임포트
# from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split  # sklearn.model_selection

# Pandas 라이브러리 임포트
import pandas as pd

### Load Data

In [5]:
# Malimg 데이터 읽어들이기

malimg = data
# malimg

malware sample에 대한 vector값

In [6]:
data = malimg[0]
data

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [7]:
data.shape

(9339, 50176)

* 정리
    * 9339개의 Malware Image Sample 데이터
    * 각각의 데이터 샘플은 50176 vector로 이루어져있다. (224x224)


## #2. Data Preprocessing

* NumPy to DataFrame 
* Data Preprocessing

### NumPy to DataFrame
malimg data타입은 NumPy배열이므로 한 번에 내용을 파악하기 어렵다.
pandas library를 이용해 데이터를 파악해보자.

In [10]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,50166,50167,50168,50169,50170,50171,50172,50173,50174,50175
0,94.0,32.0,2.0,3.0,0.0,98.0,40.0,30.0,4.0,6.0,...,111.0,170.0,155.0,138.0,155.0,150.0,75.0,129.0,102.0,124.0
1,54.0,29.0,6.0,35.0,49.0,147.0,53.0,50.0,1.0,43.0,...,102.0,78.0,132.0,115.0,147.0,96.0,112.0,106.0,119.0,100.0
2,54.0,29.0,6.0,35.0,49.0,147.0,53.0,50.0,1.0,43.0,...,145.0,103.0,85.0,126.0,112.0,120.0,108.0,120.0,126.0,104.0
3,94.0,32.0,2.0,3.0,0.0,98.0,40.0,30.0,4.0,6.0,...,91.0,186.0,160.0,118.0,161.0,170.0,129.0,100.0,92.0,133.0
4,95.0,32.0,2.0,3.0,0.0,98.0,40.0,31.0,4.0,6.0,...,79.0,94.0,173.0,113.0,113.0,107.0,116.0,101.0,126.0,129.0


feature의 개수가 몇 개 안될때는 확인이 가능하지만 이미지 데이터처럼 feature의 개수가 50172개일 경우에는 효과적인지 모르겠다.

In [8]:
# 목적변수 데이터 출력
target = malimg[1]
target

array([ 0.,  0.,  0., ..., 24., 24., 24.])

### Data Preprocessing
* 설명변수 목적변수 분할
* Train_Test_Split

Q. 이미 데이터 전처리를 통해 설명변수와 목적변수를 분할해 두었다.

In [12]:
data

array([[ 94.,  32.,   2., ..., 129., 102., 124.],
       [ 54.,  29.,   6., ..., 106., 119., 100.],
       [ 54.,  29.,   6., ..., 120., 126., 104.],
       ...,
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.],
       [ 65.,  22.,  59., ..., 117., 116.,  98.]])

In [13]:
target

array([ 0.,  0.,  0., ..., 24., 24., 24.])

### Train_Test Split

데이터 집합을 훈련 데이터와 테스트 데이터로 분할한다.

In [9]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = train_test_split(data, target, test_size = 0.2)

print(len(train_X))  # train data에서 설명변수X
print(len(test_X))   # test data에서 설명변수X

7471
1868


Train_Test Split 정리<br>
train 데이터의 설명변수의 변수명은 train_X, 목적변수의 변수명은 train_Y
test 데이터에서는?<br>
test 데이터의 설명변수의 변수명은 test_X, 목적변수명은 test_Y로 한다.

## #3. Make Tensor(텐서 생성)

완성된 train, test 데이터를 PyTorch의 tensor 데이터로 변환해준다.<br>
즉 모델의 학습 준비가 끝난 데이터를 파이토치가 다룰 수 있는 형태로 정리한다.
* 훈련 데이터, 테스트 데이터 tensor 변환
* train데이터의 X와 Y데이터 tensorDataset으로 병합
* Mini-Batch


### 훈련 데이터, 테스트 데이터 tensor 변환
<code>torch.from_numpy()</code>

In [10]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

torch.Size([7471, 50176])
torch.Size([7471])
torch.Size([1868, 50176])
torch.Size([1868])


여기까지 Train_Test Split --20.04.16.Thur.pm 4:14--

git ignore *.pkl로 백업을 해놨다.<br>
주어진 데이터로 신나게 실험을 해보자!

---
훈련 데이터와 테스트 데이터를 Tensor로 변환하는 작업까지 했다. 이어서 나머지 과정들을 진행해보도록 하자.

### 설명변수와 목적변수 병합 

In [16]:
# 설명변수와 목적변수의 텐서를 합침
train = TensorDataset(train_X, train_Y)

# 텐서의 첫 번째 데이터 내용 확인
print(train[0])

(tensor([ 86., 134.,  72.,  ..., 109.,  49., 227.]), tensor(2))


### Mini-Batch 분할

In [17]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)

## #4. 신경망 구성

학습에 사용할 신경망을 구성한다.<br>
입력층의 노드 수는 50172개(설명 변수의 개수), 중간층 노드의 수는 96개, 출력층 노드의 수는 25개이다(목적변수의 수)

In [18]:
# 신경망 구성

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(50176, 96)
        self.fc2 = nn.Linear(96, 25)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

## #5. 모형 학습

In [19]:
# model이라는 이름으로 클래스의 인스턴스를 생성한다.

model = Net()

### 5-1. Optimizer SGD

In [ ]:
# 오차함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(100):
    total_loss = 0
    # 분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이터
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


--- 
Use GPU

### 참고할만한 코드

장유영 git blog: https://greeksharifa.github.io/pytorch/2018/11/10/pytorch-usage-03-How-to-Use-PyTorch/#cuda-use-gpu

In [20]:
torch.cuda.is_available()

True

In [22]:
torch.cuda.device()

TypeError: __init__() missing 1 required positional argument: 'device'

PyTorch- CUDA(GPU)사용하기<br>
https://yonghyuc.wordpress.com/2019/08/06/pytorch-cuda-gpu-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0/

머신러닝 학습시 지금 CPU를 사용하고 있는지 GPU를 사용하고 있는지 확인이 필요할 때가 있다. 나의 경우 모델을 학습시킬 때 CPU를 사용하다보니 컴퓨터가 감당을 못하는 짜릿한 경험을 하게 되었다.(20.04.16.pm5:18)

In [23]:
# Returns a bool indicating if CUDA is currently available.
torch.cuda.is_available()

True

In [24]:
# Return the index of a currently selected device.
torch.cuda.current_device()

0

In [25]:
# Return the number of GPUs available.
torch.cuda.device_count()

1

In [26]:
# Gets the name of a device.
torch.cuda.get_device_name(0)

'GeForce GTX 1050 Ti'

In [27]:
# Context-manager that changes the selected device.
# device (torch.device or int) - device index to select.
torch.cuda.device(0)

#### Tensor를 GPU에 loading하는 방법

Tensor 생성 후 해당 tensor를 GPU에 loading하기 위해 tensor를 정의 (또는 변형)하면 된다.

In [29]:
# Default CUDA device
cuda = torch.device('cuda')

# allocates a tensor on default GPU
a = torch.tensor([1., 2.], device=cuda)

# transfers a tensor from 'C'PU to 'G'PU
b = torch.tensor([1., 2.]).cuda()

# Same with .cuda()
b2= torch.tensor([1., 2.]).to(device=cuda)

#### Model의 모든 파라미터(parameter)를 GPU에 loading

In [32]:
model = Net()
model = model.cuda()

GPU에 load한 tensor는 동일하게 GPU에 load된 tensor끼리만 연산이 가능하다<br>
model 또한 parameter를 cuda로 설정한 뒤 CPU에 loading된 tensor를 넣으면 error가 발생한다.

사용하지 않는 tensor를 GPU에서 release하고 싶다면

In [33]:
torch.cuda.empty_cache()

## Do it
Tensor를 GPU에 Loading하고 Model의 모든 파라미터를 GPU에 loading

**example**

In [19]:
train_X.cuda()

tensor([[ 17.,   6.,   0.,  ...,  23.,  39.,  34.],
        [ 21.,   0.,   0.,  ...,  44.,  38.,  33.],
        [ 47.,  69.,  57.,  ..., 126., 109., 143.],
        ...,
        [ 85., 134.,  72.,  ...,  99.,  79.,  46.],
        [ 65.,  22.,  59.,  ..., 117., 116.,  98.],
        [ 93.,  82.,  70.,  ..., 147., 103., 104.]], device='cuda:0')

In [20]:
train_X

tensor([[ 17.,   6.,   0.,  ...,  23.,  39.,  34.],
        [ 21.,   0.,   0.,  ...,  44.,  38.,  33.],
        [ 47.,  69.,  57.,  ..., 126., 109., 143.],
        ...,
        [ 85., 134.,  72.,  ...,  99.,  79.,  46.],
        [ 65.,  22.,  59.,  ..., 117., 116.,  98.],
        [ 93.,  82.,  70.,  ..., 147., 103., 104.]], device='cuda:0')

In [11]:
# train data GPU loading
train_X = train_X.cuda()
train_Y = train_Y.cuda()

# test data GPU loading
test_X = test_X.cuda()
test_Y = test_Y.cuda()

# GPU로딩한 데이터 불러오기
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

torch.Size([7471, 50176])
torch.Size([7471])
torch.Size([1868, 50176])
torch.Size([1868])


### 설명변수와 목적 변수 병합

In [12]:
# train data의 설명변수와 목적변수의 텐서를 합친다
train = TensorDataset(train_X, train_Y)
print(train[0])

(tensor([17.,  6.,  0.,  ..., 23., 39., 34.], device='cuda:0'), tensor(22, device='cuda:0'))


In [21]:
print(train[1])

(tensor([21.,  0.,  0.,  ..., 44., 38., 33.], device='cuda:0'), tensor(22, device='cuda:0'))


device = 'cuda:0'

### Mini-Batch 분할

In [13]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)

## #4. 신경망 구성

학습에 사용할 신경망 구성<br>
입력층 노드 수는 50176, 중간층 노드의 수는 96, 출력층 노드의 수는 25개로 구성

In [22]:
# 신경망 구성

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(50176, 96)
        self.fc2 = nn.Linear(96, 25)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

## #5. 모형 학습

In [24]:
# 모델 인스턴스 생성
model = Net()
model = model.cuda()  # 모델의 모든 파라미터를 GPU에 로딩

# 오차함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(100):
    total_loss = 0
    # 분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이터
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)


C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


50 tensor(1175.0011, device='cuda:0')
100 tensor(1175.0145, device='cuda:0')


역시 GOD PU<br>
300 Epoch은 어떨까

In [25]:
# 모델 인스턴스 생성
model = Net()
model = model.cuda()  # 모델의 모든 파라미터를 GPU에 로딩

# 오차함수 객체 생성
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    # 분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이터
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)


C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


50 tensor(1175.0249, device='cuda:0')
100 tensor(1175.0103, device='cuda:0')
150 tensor(1175.0031, device='cuda:0')


KeyboardInterrupt: 

다중 퍼셉트론

In [26]:
# 신경망 구성

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(50176, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)

In [28]:
# 인스턴스 생성
model = Net()
model = model.cuda()

# 오차함수 객체 
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    #분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 게산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)  # 와우... 정말 간단하네...
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산 
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50회마다 누적 오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 tensor(1176.5624, device='cuda:0')
100 tensor(1176.5521, device='cuda:0')
150 tensor(1176.6886, device='cuda:0')
200 tensor(1176.5984, device='cuda:0')
250 tensor(1176.4879, device='cuda:0')
300 tensor(1176.5933, device='cuda:0')


In [29]:
# 인스턴스 생성
model = Net()
model = model.cuda()

# 오차함수 객체 
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    #분할해둔 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 게산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)  # 와우... 정말 간단하네...
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산 
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 10회마다 누적 오차 출력
    if (epoch+1) % 10 == 0:
        print(epoch+1, total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


10 tensor(1176.3198, device='cuda:0')
20 tensor(1176.4226, device='cuda:0')
30 tensor(1176.5176, device='cuda:0')
40 tensor(1176.7001, device='cuda:0')
50 tensor(1176.6053, device='cuda:0')
60 tensor(1176.4613, device='cuda:0')
70 tensor(1176.6798, device='cuda:0')
80 tensor(1176.5718, device='cuda:0')
90 tensor(1176.6805, device='cuda:0')
100 tensor(1176.4675, device='cuda:0')
110 tensor(1176.6964, device='cuda:0')
120 tensor(1176.6578, device='cuda:0')
130 tensor(1176.7168, device='cuda:0')
140 tensor(1176.7245, device='cuda:0')
150 tensor(1176.7758, device='cuda:0')
160 tensor(1176.5477, device='cuda:0')
170 tensor(1176.5021, device='cuda:0')
180 tensor(1176.5742, device='cuda:0')
190 tensor(1176.4723, device='cuda:0')
200 tensor(1176.7435, device='cuda:0')
210 tensor(1176.6791, device='cuda:0')
220 tensor(1176.4398, device='cuda:0')
230 tensor(1176.6129, device='cuda:0')
240 tensor(1176.5859, device='cuda:0')
250 tensor(1176.6913, device='cuda:0')
260 tensor(1176.6699, device='cuda

GPU로 학습은 잘 했으나 결과가 너무나도 좋지 않다. loss가 줄어들 기미를 보이지 않는다.

* 원인 분석
    * 첫 째, 이미지 데이터를 쭉 펼쳐놨기에 형상이 무시되었고 차원이 너무 크다
    * 둘 째, 앞선 문제점과 비슷하다. 데이터의 형상이 무시된 상태에서 고차원의 데이터를 단순하게 분류하려 하였다.

1000 epoch을 학습해도 결과가 좋을 것 같지는 않다...(혹시 모르니깐 자기 전에 학습을 시켜놔보긴 하겠지만..)